<a href="https://colab.research.google.com/github/MariaShaiina/DV_SSau_2023/blob/main/lab4/DV_L4_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Выполнила: Шаина Мария Михайловна
---
гр. 6231-010402D

# Преобразование данных в упорядоченную форму (tidy data)

# Что такое упорядоченные (tidy) данные?

Hadley в работе (http://vita.had.co.nz/papers/tidy-data.pdf) предлагает три основных принципа, определяющих, является ли набор данных упорядоченным:

1. Каждая переменная формирует столбец.

2. Каждое наблюдение формирует строку.

3. Каждый тип наблюдения формирует таблицу.

In [1]:
import pandas as pd
import numpy as np

from google.colab import drive, data_table

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Упорядочивание данных со значениями переменных в заголовках с использованием `stack()`

Чтобы понять разницу между упорядоченными (tidy) и неупорядоченными (messy) данными, рассмотрим следующую таблицу.

In [3]:
state_fruit = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/state_fruit.csv', index_col=0)
state_fruit

,Apple,Orange,Banana
Texas,12,10,40
Arizona,9,7,12
Florida,0,14,190


Кажется, что в этих данных нет ничего неупорядоченного, а информация из неё легко извлекается. На самом же деле, эти данные не вполне упорядочены. Каждый столбец на самом деле – это значение переменной. Более того, в этой таблице вообще нет имён переменных!

Одним из первых шагов при преобразовании данных в упорядоченный вид: определение всех переменных. В этом наборе данных это будут *штат* и *фрукт*. Кроме того, в этой таблице есть численные значения, которые вообще никак не определены, поэтому обозначим эту переменную как *вес*.

Сначала превратим столбцы в значения переменной *фрукт*.

In [4]:
state_fruit.stack()

Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

Сейчас эта таблица многоиндексная (MultiIndex). Фактически, каждое значение *будущей* переменной **штат** представляет собой целую таблицу. Сбросим лишние индексы.

In [5]:
state_fruit_tidy = state_fruit.stack().reset_index()
state_fruit_tidy

,level_0,level_1,0
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


И определим названия столбцов.

In [6]:
state_fruit_tidy.columns = ['state', 'fruit', 'weight']
state_fruit_tidy

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


В целом, это можно было сделать чуть иначе. Но результат будет тот же.

In [7]:
state_fruit.stack() \
           .rename_axis(['state', 'fruit'])

state    fruit 
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

Сброс индексов в любом случае будет нужен

In [8]:
state_fruit.stack()\
           .rename_axis(['state', 'fruit'])\
           .reset_index(name='weight')

,state,fruit,weight
0,Texas,Apple,12
1,Texas,Orange,10
2,Texas,Banana,40
3,Arizona,Apple,9
4,Arizona,Orange,7
5,Arizona,Banana,12
6,Florida,Apple,0
7,Florida,Orange,14
8,Florida,Banana,190


Обратите внимание, что `stack()` не преобразует **только** столбцы индекса. Это значит, что если штаты будут в не-индексном столбце, мы получим совсем не то, что планировали.

In [9]:
state_fruit2 = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/state_fruit2.csv')
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


И заголовок **штат** уходит в значения...

In [10]:
state_fruit2.stack()

0  State       Texas
   Apple          12
   Orange         10
   Banana         40
1  State     Arizona
   Apple           9
   Orange          7
   Banana         12
2  State     Florida
   Apple           0
   Orange         14
   Banana        190
dtype: object

Этого можно избежать, явно задав этот столбец индексным.

In [11]:
state_fruit2.set_index('State').stack()

State          
Texas    Apple      12
         Orange     10
         Banana     40
Arizona  Apple       9
         Orange      7
         Banana     12
Florida  Apple       0
         Orange     14
         Banana    190
dtype: int64

# Упорядочивание данных со значениями переменных в заголовках с использованием `melt()`

In [12]:
state_fruit2 = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/state_fruit2.csv')
state_fruit2

,State,Apple,Orange,Banana
0,Texas,12,10,40
1,Arizona,9,7,12
2,Florida,0,14,190


In [13]:
state_fruit2.melt(id_vars=['State'],
                 value_vars=['Apple', 'Orange', 'Banana'])

,State,variable,value
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


И у нас сразу получилась правильная таблица. `melt()` превращает названия столбцов из `value_vars` в значения переменной с названием `variable`, а их значения в значения переменной с названием `value`

Можно сразу расставить названия столбцов

In [14]:
state_fruit2.melt(id_vars=['State'],
                 value_vars=['Apple', 'Orange', 'Banana'],
                 var_name='Fruit',
                 value_name='Weight')

,State,Fruit,Weight
0,Texas,Apple,12
1,Arizona,Apple,9
2,Florida,Apple,0
3,Texas,Orange,10
4,Arizona,Orange,7
5,Florida,Orange,14
6,Texas,Banana,40
7,Arizona,Banana,12
8,Florida,Banana,190


Разумеется, применив `melt()` с параметрами по умолчанию мы опять получим что-то странное.

In [15]:
state_fruit2.melt()

,variable,value
0,State,Texas
1,State,Arizona
2,State,Florida
3,Apple,12
4,Apple,9
5,Apple,0
6,Orange,10
7,Orange,7
8,Orange,14
9,Banana,40


# `stack()` для нескольких групп переменных одновременно

In [16]:
movie = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/movie.csv')
actor = movie[['movie_title', 'actor_1_name', 'actor_2_name', 'actor_3_name',
               'actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes']]
actor.head()

,movie_title,actor_1_name,actor_2_name,actor_3_name,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes
0,Avatar,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


Немного переименуем заголовки. Нам это пригодится дальше.

In [17]:
def change_col_name(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = col_name[:5] + col_name[fb_idx - 1:] + col_name[5:fb_idx-1]
    return col_name

In [18]:
actor2 = actor.rename(columns=change_col_name)
actor2.head()

,movie_title,actor_1,actor_2,actor_3,actor_facebook_likes_1,actor_facebook_likes_2,actor_facebook_likes_3
0,Avatar,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


Так лучше. Обратите внимание, что столбцы, которые мы хотим "сгруппировать", одинаково начинаются и оканчиваются различными числами. Это необходимо для работы функции `wide_to_long()`

In [19]:
stubs = ['actor', 'actor_facebook_likes']
actor2_tidy = pd.wide_to_long(actor2,
                              stubnames=stubs,
                              i=['movie_title'],
                              j='actor_num',
                              sep='_').reset_index()
actor2_tidy.head()

,movie_title,actor_num,actor,actor_facebook_likes
0,Avatar,1,CCH Pounder,1000.0
1,Pirates of the Caribbean: At World's End,1,Johnny Depp,40000.0
2,Spectre,1,Christoph Waltz,11000.0
3,The Dark Knight Rises,1,Tom Hardy,27000.0
4,Star Wars: Episode VII - The Force Awakens,1,Doug Walker,131.0


Вы потрясающие!

Ещё один пример

In [20]:
df = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/stackme.csv')
df

,State,Country,a1,b2,Test,d,e
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


In [21]:
df2 = df.rename(columns = {'a1':'group1_a1', 'b2':'group1_b2',
                           'd':'group2_a1', 'e':'group2_b2'})
df2

,State,Country,group1_a1,group1_b2,Test,group2_a1,group2_b2
0,TX,US,0.45,0.3,Test1,2,6
1,MA,US,0.03,1.2,Test2,9,7
2,ON,CAN,0.70,4.2,Test3,4,2


Кстати, можно сразу же переименовать всё, что захотите

In [22]:
pd.wide_to_long(df2,
                stubnames=['group1', 'group2'],
                i=['State', 'Country', 'Test'],
                j='Label',
                suffix='.+',
                sep='_')

group1  group2
State Country Test  Label                
TX    US      Test1 a1       0.45       2
                    b2       0.30       6
MA    US      Test2 a1       0.03       9
                    b2       1.20       7
ON    CAN     Test3 a1       0.70       4
                    b2       4.20       2

# Использование `unstack()` и `pivot()`

In [23]:
usecol_func = lambda x: 'UGDS_' in x or x == 'INSTNM'
college = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/college.csv',
                          index_col='INSTNM',
                          usecols=usecol_func)
college.head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [24]:
college_stacked = college.stack()
college_stacked.head(18)

INSTNM                                         
Alabama A & M University             UGDS_WHITE    0.0333
                                     UGDS_BLACK    0.9353
                                     UGDS_HISP     0.0055
                                     UGDS_ASIAN    0.0019
                                     UGDS_AIAN     0.0024
                                     UGDS_NHPI     0.0019
                                     UGDS_2MOR     0.0000
                                     UGDS_NRA      0.0059
                                     UGDS_UNKN     0.0138
University of Alabama at Birmingham  UGDS_WHITE    0.5922
                                     UGDS_BLACK    0.2600
                                     UGDS_HISP     0.0283
                                     UGDS_ASIAN    0.0518
                                     UGDS_AIAN     0.0022
                                     UGDS_NHPI     0.0007
                                     UGDS_2MOR     0.0368
                        

In [25]:
college_stacked.unstack().head()

,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
INSTNM,,,,,,,,,
Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [26]:
college2 = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/college.csv',
                      usecols=usecol_func)
college2.head()

,INSTNM,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN
0,Alabama A & M University,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138
1,University of Alabama at Birmingham,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100
2,Amridge University,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715
3,University of Alabama in Huntsville,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350
4,Alabama State University,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137


In [27]:
college_melted = college2.melt(id_vars='INSTNM',
                               var_name='Race',
                               value_name='Percentage')
college_melted.head()

,INSTNM,Race,Percentage
0,Alabama A & M University,UGDS_WHITE,0.0333
1,University of Alabama at Birmingham,UGDS_WHITE,0.5922
2,Amridge University,UGDS_WHITE,0.2990
3,University of Alabama in Huntsville,UGDS_WHITE,0.6988
4,Alabama State University,UGDS_WHITE,0.0158


In [28]:
melted_inv = college_melted.pivot(index='INSTNM',
                                  columns='Race',
                                  values='Percentage')
melted_inv.head()

Race,UGDS_2MOR,UGDS_AIAN,UGDS_ASIAN,UGDS_BLACK,UGDS_HISP,UGDS_NHPI,UGDS_NRA,UGDS_UNKN,UGDS_WHITE
INSTNM,,,,,,,,,
A & W Healthcare Educators,0.0000,0.0,0.0000,0.9750,0.0250,0.0,0.0000,0.0000,0.0000
A T Still University of Health Sciences,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABC Beauty Academy,0.0000,0.0,0.9333,0.0333,0.0333,0.0,0.0000,0.0000,0.0000
ABC Beauty College Inc,0.0000,0.0,0.0000,0.6579,0.0526,0.0,0.0000,0.0000,0.2895
AI Miami International University of Art and Design,0.0018,0.0,0.0018,0.0198,0.4773,0.0,0.0025,0.4644,0.0324


А если мы хотим прежний порядок столбцов то используем

In [29]:
college2_replication = melted_inv.loc[college2['INSTNM'],
                                      college2.columns[1:]]\
                                         .reset_index()
college2.equals(college2_replication)

True

Таким способом можно транспонировать данные. Воспользуемся тем, что `unstack()` можно делать как по внутреннему индексу (1), так и по внешнему (0). Кстати, по умолчанию он берёт *самый внутренний* индекс (при этом параметр равен -1)

In [30]:
college.stack().unstack(0)

INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Athens State University,Auburn University at Montgomery,Auburn University,...,MCI Institute of Technology-Boca Raton,West Coast University-Miami,National American University-Houston,Aparicio-Levy Technical College,Fred D. Learey Technical College,Hollywood Institute of Beauty Careers-West Palm Beach,Hollywood Institute of Beauty Careers-Casselberry,Coachella Valley Beauty College-Beaumont,Dewey University-Mayaguez,Coastal Pines Technical College
UGDS_WHITE,0.0333,0.5922,0.2990,0.6988,0.0158,0.7825,0.7255,0.7823,0.5328,0.8507,...,0.0199,0.1522,0.1858,0.2431,0.3731,0.2182,0.1200,0.3284,0.0,0.6762
UGDS_BLACK,0.9353,0.2600,0.4192,0.1255,0.9208,0.1119,0.2613,0.1200,0.3376,0.0704,...,0.2815,0.1739,0.6443,0.1215,0.1388,0.4182,0.3333,0.1045,0.0,0.2508
UGDS_HISP,0.0055,0.0283,0.0069,0.0382,0.0121,0.0348,0.0044,0.0191,0.0074,0.0248,...,0.6854,0.6087,0.0672,0.6243,0.3080,0.2364,0.4400,0.4925,1.0,0.0359
UGDS_ASIAN,0.0019,0.0518,0.0034,0.0376,0.0019,0.0106,0.0025,0.0053,0.0221,0.0227,...,0.0132,0.0217,0.0079,0.0055,0.0000,0.0182,0.0000,0.0149,0.0,0.0045
UGDS_AIAN,0.0024,0.0022,0.0000,0.0143,0.0010,0.0038,0.0044,0.0157,0.0044,0.0074,...,0.0000,0.0000,0.0079,0.0055,0.0000,0.0000,0.0000,0.0299,0.0,0.0034
UGDS_NHPI,0.0019,0.0007,0.0000,0.0002,0.0006,0.0009,0.0000,0.0010,0.0016,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0149,0.0,0.0017
UGDS_2MOR,0.0000,0.0368,0.0000,0.0172,0.0098,0.0261,0.0000,0.0174,0.0297,0.0000,...,0.0000,0.0435,0.0751,0.0000,0.0022,0.0000,0.0400,0.0149,0.0,0.0191
UGDS_NRA,0.0059,0.0179,0.0000,0.0332,0.0243,0.0268,0.0000,0.0057,0.0397,0.0100,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0182,0.0000,0.0000,0.0,0.0028
UGDS_UNKN,0.0138,0.0100,0.2715,0.0350,0.0137,0.0026,0.0019,0.0334,0.0246,0.0140,...,0.0000,0.0000,0.0119,0.0000,0.1779,0.0909,0.0667,0.0000,0.0,0.0056


In [31]:
college.T

INSTNM,Alabama A & M University,University of Alabama at Birmingham,Amridge University,University of Alabama in Huntsville,Alabama State University,The University of Alabama,Central Alabama Community College,Athens State University,Auburn University at Montgomery,Auburn University,...,Strayer University-North Dallas,Strayer University-San Antonio,Strayer University-Stafford,WestMed College - Merced,Vantage College,SAE Institute of Technology San Francisco,Rasmussen College - Overland Park,National Personal Training Institute of Cleveland,Bay Area Medical Academy - San Jose Satellite Location,Excel Learning Center-San Antonio South
UGDS_WHITE,0.0333,0.5922,0.2990,0.6988,0.0158,0.7825,0.7255,0.7823,0.5328,0.8507,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_BLACK,0.9353,0.2600,0.4192,0.1255,0.9208,0.1119,0.2613,0.1200,0.3376,0.0704,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_HISP,0.0055,0.0283,0.0069,0.0382,0.0121,0.0348,0.0044,0.0191,0.0074,0.0248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_ASIAN,0.0019,0.0518,0.0034,0.0376,0.0019,0.0106,0.0025,0.0053,0.0221,0.0227,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_AIAN,0.0024,0.0022,0.0000,0.0143,0.0010,0.0038,0.0044,0.0157,0.0044,0.0074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NHPI,0.0019,0.0007,0.0000,0.0002,0.0006,0.0009,0.0000,0.0010,0.0016,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_2MOR,0.0000,0.0368,0.0000,0.0172,0.0098,0.0261,0.0000,0.0174,0.0297,0.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_NRA,0.0059,0.0179,0.0000,0.0332,0.0243,0.0268,0.0000,0.0057,0.0397,0.0100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UGDS_UNKN,0.0138,0.0100,0.2715,0.0350,0.0137,0.0026,0.0019,0.0334,0.0246,0.0140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# `unstack()` после объединения `groupby`

In [32]:
employee = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/employee.csv')
employee

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22
...,...,...,...,...,...,...,...,...,...,...
1995,1995,POLICE OFFICER,Houston Police Department-HPD,43443.0,White,Full Time,Male,Active,2014-06-09,2015-06-09
1996,1996,COMMUNICATIONS CAPTAIN,Houston Fire Department (HFD),66523.0,Black or African American,Full Time,Male,Active,2003-09-02,2013-10-06
1997,1997,POLICE OFFICER,Houston Police Department-HPD,43443.0,White,Full Time,Male,Active,2014-10-13,2015-10-13
1998,1998,POLICE OFFICER,Houston Police Department-HPD,55461.0,Asian/Pacific Islander,Full Time,Male,Active,2009-01-20,2011-07-02


Сгруппируем по расе

In [33]:
employee.groupby('RACE')['BASE_SALARY'].mean().astype(int)

RACE
American Indian or Alaskan Native    60272
Asian/Pacific Islander               61660
Black or African American            50137
Hispanic/Latino                      52345
Others                               51278
White                                64419
Name: BASE_SALARY, dtype: int64

Здесь всё просто и никакого `reshape()` не требуется

А если сгруппировать по расе и полу

In [34]:
agg = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'].mean().astype(int)
agg

RACE                               GENDER
American Indian or Alaskan Native  Female    60238
                                   Male      60305
Asian/Pacific Islander             Female    63226
                                   Male      61033
Black or African American          Female    48915
                                   Male      51082
Hispanic/Latino                    Female    46503
                                   Male      54782
Others                             Female    63785
                                   Male      38771
White                              Female    66793
                                   Male      63940
Name: BASE_SALARY, dtype: int64

Такая группировка сложнее и должна быть пересортирована для упрощения сравнения

Сделаем из неё не-упорядоченные данные, которые, тем не менее, удобнее для визуального восприятия

In [35]:
agg.unstack('GENDER')

GENDER,Female,Male
RACE,,
American Indian or Alaskan Native,60238,60305
Asian/Pacific Islander,63226,61033
Black or African American,48915,51082
Hispanic/Latino,46503,54782
Others,63785,38771
White,66793,63940


Или так:

In [36]:
agg.unstack('RACE')

RACE,American Indian or Alaskan Native,Asian/Pacific Islander,Black or African American,Hispanic/Latino,Others,White
GENDER,,,,,,
Female,60238,63226,48915,46503,63785,66793
Male,60305,61033,51082,54782,38771,63940


Если столцов-агрегаторов будет несколько, мы сразу получим датафрейм.

In [37]:
agg2 = employee.groupby(['RACE', 'GENDER'])['BASE_SALARY'].agg(['mean', 'max', 'min']).astype(int)
agg2

mean     max    min
RACE                              GENDER                      
American Indian or Alaskan Native Female  60238   98536  26125
                                  Male    60305   81239  26125
Asian/Pacific Islander            Female  63226  130416  26125
                                  Male    61033  163228  27914
Black or African American         Female  48915  150416  24960
                                  Male    51082  275000  26125
Hispanic/Latino                   Female  46503  126115  26125
                                  Male    54782  165216  26104
Others                            Female  63785   63785  63785
                                  Male    38771   38771  38771
White                             Female  66793  178331  27955
                                  Male    63940  210588  26125

`unstack()` по полу даст многоиндексные столбцы

In [38]:
agg2.unstack('GENDER')

mean            max            min       
GENDER                            Female   Male  Female    Male Female   Male
RACE                                                                         
American Indian or Alaskan Native  60238  60305   98536   81239  26125  26125
Asian/Pacific Islander             63226  61033  130416  163228  26125  27914
Black or African American          48915  51082  150416  275000  24960  26125
Hispanic/Latino                    46503  54782  126115  165216  26125  26104
Others                             63785  38771   63785   38771  63785  38771
White                              66793  63940  178331  210588  27955  26125

# Повторение функционала pivot_table с использованием агрегатора `groupby`

In [39]:
flights = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/flights.csv')
flights.head()

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [40]:
fp = flights.pivot_table(index='AIRLINE',
                         columns='ORG_AIR',
                         values='CANCELLED',
                         aggfunc='sum',
                         fill_value=0).round(2)
fp.head()

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2
AS,0,0,0,0,0,0,0,0,0,0
B6,0,0,0,0,0,0,0,0,0,1
DL,28,1,0,0,1,1,4,0,1,2
EV,18,6,27,36,0,0,6,53,0,0


In [41]:
fg = flights.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum()
fg.head()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
Name: CANCELLED, dtype: int64

In [42]:
fg_unstack = fg.unstack('ORG_AIR', fill_value=0)
fg_unstack.head()

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO
AIRLINE,,,,,,,,,,
AA,3,4,86,3,3,11,3,35,4,2
AS,0,0,0,0,0,0,0,0,0,0
B6,0,0,0,0,0,0,0,0,0,1
DL,28,1,0,0,1,1,4,0,1,2
EV,18,6,27,36,0,0,6,53,0,0


In [43]:
fg.equals(fg_unstack)

False

# Упорядочивание (tidying) данных, когда несколько переменных хранятся как заголовки столбцов

In [44]:
weightlifting = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/weightlifting_men.csv')
weightlifting

,Weight Category,M35 35-39,M40 40-44,M45 45-49,M50 50-54,M55 55-59,M60 60-64,M65 65-69,M70 70-74,M75 75-79,M80 80+
0,56,137,130,125,115,102,92,80,67,62,55
1,62,152,145,137,127,112,102,90,75,67,57
2,69,167,160,150,140,125,112,97,82,75,60
3,77,182,172,165,150,135,122,107,90,82,65
4,85,192,182,175,160,142,130,112,95,87,70
5,94,202,192,182,167,150,137,120,100,90,75
6,105,210,200,190,175,157,142,122,102,95,80
7,105+,217,207,197,182,165,150,127,107,100,85


In [45]:
wl_melt = weightlifting.melt(id_vars='Weight Category',
                             var_name='sex_age',
                             value_name='Qual Total')
wl_melt.head()

,Weight Category,sex_age,Qual Total
0,56,M35 35-39,137
1,62,M35 35-39,152
2,69,M35 35-39,167
3,77,M35 35-39,182
4,85,M35 35-39,192


In [46]:
sex_age = wl_melt['sex_age'].str.split(expand=True)
sex_age.head()

,0,1
0,M35,35-39
1,M35,35-39
2,M35,35-39
3,M35,35-39
4,M35,35-39


In [47]:
sex_age.columns = ['Sex', 'Age Group']
sex_age.head()

,Sex,Age Group
0,M35,35-39
1,M35,35-39
2,M35,35-39
3,M35,35-39
4,M35,35-39


In [48]:
sex_age['Sex'] = sex_age['Sex'].str[0]
sex_age.head()

,Sex,Age Group
0,M,35-39
1,M,35-39
2,M,35-39
3,M,35-39
4,M,35-39


In [49]:
wl_cat_total = wl_melt[['Weight Category', 'Qual Total']]
wl_tidy = pd.concat([sex_age, wl_cat_total], axis='columns')
wl_tidy.head()

,Sex,Age Group,Weight Category,Qual Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192


In [50]:
cols = ['Weight Category', 'Qual Total']
sex_age[cols] = wl_melt[cols]
sex_age.head()

,Sex,Age Group,Weight Category,Qual Total
0,M,35-39,56,137
1,M,35-39,62,152
2,M,35-39,69,167
3,M,35-39,77,182
4,M,35-39,85,192


Для любителей регулярных выражений

In [51]:
age_group = wl_melt.sex_age.str.extract('(\d{2}[-+](?:\d{2})?)', expand=False)
sex = wl_melt.sex_age.str[0]
new_cols = {'Sex':sex,
            'Age Group': age_group}

In [52]:
wl_tidy2 = wl_melt.assign(**new_cols).drop('sex_age', axis='columns')
wl_tidy2.head()

,Weight Category,Qual Total,Sex,Age Group
0,56,137,M,35-39
1,62,152,M,35-39
2,69,167,M,35-39
3,77,182,M,35-39
4,85,192,M,35-39


# Упорядочивание данных с несколькими переменными, хранящимися в форме значений столбцов

In [53]:
inspections = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/restaurant_inspections.csv', parse_dates=['Date'])
inspections.head(10)

,Name,Date,Info,Value
0,E & E Grill House,2017-08-08,Borough,MANHATTAN
1,E & E Grill House,2017-08-08,Cuisine,American
2,E & E Grill House,2017-08-08,Description,Non-food contact surface improperly constructe...
3,E & E Grill House,2017-08-08,Grade,A
4,E & E Grill House,2017-08-08,Score,9.0
5,PIZZA WAGON,2017-04-12,Borough,BROOKLYN
6,PIZZA WAGON,2017-04-12,Cuisine,Pizza
7,PIZZA WAGON,2017-04-12,Description,"Food contact surface not properly washed, rins..."
8,PIZZA WAGON,2017-04-12,Grade,A
9,PIZZA WAGON,2017-04-12,Score,10.0


In [54]:
inspections.pivot(index=['Name', 'Date'], columns='Info', values='Value')

,Info,Borough,Cuisine,Description,Grade,Score
Name,Date,,,,,
3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0
...,...,...,...,...,...,...
VALL'S PIZZERIA,2017-03-15,STATEN ISLAND,Pizza/Italian,Wiping cloths soiled or not stored in sanitizi...,A,9.0
VIP GRILL,2017-06-12,BROOKLYN,Jewish/Kosher,Hot food item not held at or above 140Âº F.,A,10.0
WAHIZZA,2017-04-13,MANHATTAN,Pizza,"No facilities available to wash, rinse and san...",A,10.0


In [55]:
inspections.set_index(['Name','Date', 'Info']).head(10)

Value
Name              Date       Info                                                          
E & E Grill House 2017-08-08 Borough                                              MANHATTAN
                             Cuisine                                               American
                             Description  Non-food contact surface improperly constructe...
                             Grade                                                        A
                             Score                                                      9.0
PIZZA WAGON       2017-04-12 Borough                                               BROOKLYN
                             Cuisine                                                  Pizza
                             Description  Food contact surface not properly washed, rins...
                             Grade                                                        A
                             Score                                                     10.0

In [56]:
inspections.set_index(['Name','Date', 'Info']).unstack('Info').head()

Value  \
Info                                              Borough   
Name                                Date                    
3 STAR JUICE CENTER                 2017-05-10   BROOKLYN   
A & L PIZZA RESTAURANT              2017-08-22   BROOKLYN   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25   BROOKLYN   
ANTOJITOS DELI FOOD                 2017-06-01   BROOKLYN   
BANGIA                              2017-06-16  MANHATTAN   

                                                                                                   \
Info                                                                                      Cuisine   
Name                                Date                                                            
3 STAR JUICE CENTER                 2017-05-10                     Juice, Smoothies, Fruit Salads   
A & L PIZZA RESTAURANT              2017-08-22                                              Pizza   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25                                            Turkish   
ANTOJITOS DELI FOOD                 2017-06-01  Latin (Cuban, Dominican, Puerto Rican, South &...   
BANGIA                              2017-06-16                                             Korean   

                                                                                                   \
Info                                                                                  Description   
Name                                Date                                                            
3 STAR JUICE CENTER                 2017-05-10  Facility not vermin proof. Harborage or condit...   
A & L PIZZA RESTAURANT              2017-08-22  Facility not vermin proof. Harborage or condit...   
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25  Plumbing not properly installed or maintained;...   
ANTOJITOS DELI FOOD                 2017-06-01  Live roaches present in facility's food and/or...   
BANGIA                              2017-06-16  Covered garbage receptacle not provided or ina...   

                                                            
Info                                           Grade Score  
Name                                Date                    
3 STAR JUICE CENTER                 2017-05-10     A  12.0  
A & L PIZZA RESTAURANT              2017-08-22     A   9.0  
AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25     A  13.0  
ANTOJITOS DELI FOOD                 2017-06-01     A  10.0  
BANGIA                              2017-06-16     A   9.0

In [57]:
insp_tidy = inspections.set_index(['Name','Date', 'Info']) \
                               .unstack('Info') \
                               .reset_index(col_level=-1)
insp_tidy.head()

Value  \
Info                                 Name       Date    Borough   
0                     3 STAR JUICE CENTER 2017-05-10   BROOKLYN   
1                  A & L PIZZA RESTAURANT 2017-08-22   BROOKLYN   
2     AKSARAY TURKISH CAFE AND RESTAURANT 2017-07-25   BROOKLYN   
3                     ANTOJITOS DELI FOOD 2017-06-01   BROOKLYN   
4                                  BANGIA 2017-06-16  MANHATTAN   

                                                         \
Info                                            Cuisine   
0                        Juice, Smoothies, Fruit Salads   
1                                                 Pizza   
2                                               Turkish   
3     Latin (Cuban, Dominican, Puerto Rican, South &...   
4                                                Korean   

                                                                     
Info                                        Description Grade Score  
0     Facility not vermin proof. Harborage or condit...     A  12.0  
1     Facility not vermin proof. Harborage or condit...     A   9.0  
2     Plumbing not properly installed or maintained;...     A  13.0  
3     Live roaches present in facility's food and/or...     A  10.0  
4     Covered garbage receptacle not provided or ina...     A   9.0

In [58]:
insp_tidy.columns = insp_tidy.columns.droplevel(0).rename(None)
insp_tidy.head()

,Name,Date,Borough,Cuisine,Description,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
4,BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0


In [59]:
inspections.set_index(['Name','Date', 'Info']) \
          .squeeze() \
          .unstack('Info') \
          .reset_index() \
          .rename_axis(None, axis='columns')

,Name,Date,Borough,Cuisine,Description,Grade,Score
0,3 STAR JUICE CENTER,2017-05-10,BROOKLYN,"Juice, Smoothies, Fruit Salads",Facility not vermin proof. Harborage or condit...,A,12.0
1,A & L PIZZA RESTAURANT,2017-08-22,BROOKLYN,Pizza,Facility not vermin proof. Harborage or condit...,A,9.0
2,AKSARAY TURKISH CAFE AND RESTAURANT,2017-07-25,BROOKLYN,Turkish,Plumbing not properly installed or maintained;...,A,13.0
3,ANTOJITOS DELI FOOD,2017-06-01,BROOKLYN,"Latin (Cuban, Dominican, Puerto Rican, South &...",Live roaches present in facility's food and/or...,A,10.0
4,BANGIA,2017-06-16,MANHATTAN,Korean,Covered garbage receptacle not provided or ina...,A,9.0
...,...,...,...,...,...,...,...
95,VALL'S PIZZERIA,2017-03-15,STATEN ISLAND,Pizza/Italian,Wiping cloths soiled or not stored in sanitizi...,A,9.0
96,VIP GRILL,2017-06-12,BROOKLYN,Jewish/Kosher,Hot food item not held at or above 140Âº F.,A,10.0
97,WAHIZZA,2017-04-13,MANHATTAN,Pizza,"No facilities available to wash, rinse and san...",A,10.0
98,WANG MANDOO HOUSE,2017-08-29,QUEENS,Korean,Accurate thermometer not provided in refrigera...,A,12.0


# Подготовка данных, когда два или более значений хранятся в одной ячейке

In [60]:
cities = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/texas_cities.csv')
cities

,City,Geolocation
0,Houston,"29.7604° N, 95.3698° W"
1,Dallas,"32.7767° N, 96.7970° W"
2,Austin,"30.2672° N, 97.7431° W"


In [61]:
geolocations = cities.Geolocation.str.split(pat='. ', expand=True)
geolocations.columns = ['latitude', 'latitude direction', 'longitude', 'longitude direction']
geolocations

,latitude,latitude direction,longitude,longitude direction
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [62]:
geolocations = geolocations.astype({'latitude':'float', 'longitude':'float'})
geolocations.dtypes

latitude               float64
latitude direction      object
longitude              float64
longitude direction     object
dtype: object

In [63]:
cities_tidy = pd.concat([cities['City'], geolocations], axis='columns')
cities_tidy

,City,latitude,latitude direction,longitude,longitude direction
0,Houston,29.7604,N,95.3698,W
1,Dallas,32.7767,N,96.7970,W
2,Austin,30.2672,N,97.7431,W


In [64]:
pd.concat([cities['City'], geolocations], axis='columns')

,City,latitude,latitude direction,longitude,longitude direction
0,Houston,29.7604,N,95.3698,W
1,Dallas,32.7767,N,96.7970,W
2,Austin,30.2672,N,97.7431,W


In [65]:
temp = geolocations.apply(pd.to_numeric, errors='ignore')
temp

,latitude,latitude direction,longitude,longitude direction
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [66]:
temp.dtypes

latitude               float64
latitude direction      object
longitude              float64
longitude direction     object
dtype: object

Опять регулярные выражения

In [67]:
cities.Geolocation.str.split(pat='° |, ', expand=True)

,0,1,2,3
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


In [68]:
cities.Geolocation.str.extract('([0-9.]+). (N|S), ([0-9.]+). (E|W)', expand=True)

,0,1,2,3
0,29.7604,N,95.3698,W
1,32.7767,N,96.7970,W
2,30.2672,N,97.7431,W


# Подготовка данных, когда переменные хранятся в названиях столбцов и их значениях

In [69]:
sensors = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/sensors.csv')
sensors

,Group,Property,2012,2013,2014,2015,2016
0,A,Pressure,928,873,814,973,870
1,A,Temperature,1026,1038,1009,1036,1042
2,A,Flow,819,806,861,882,856
3,B,Pressure,817,877,914,806,942
4,B,Temperature,1008,1041,1009,1002,1013
5,B,Flow,887,899,837,824,873


In [70]:
sensors.melt(id_vars=['Group', 'Property'], var_name='Year').head(6)

,Group,Property,Year,value
0,A,Pressure,2012,928
1,A,Temperature,2012,1026
2,A,Flow,2012,819
3,B,Pressure,2012,817
4,B,Temperature,2012,1008
5,B,Flow,2012,887


In [71]:
sensors.melt(id_vars=['Group', 'Property'], var_name='Year') \
       .pivot_table(index=['Group', 'Year'], columns='Property', values='value') \
       .reset_index() \
       .rename_axis(None, axis='columns')

,Group,Year,Flow,Pressure,Temperature
0,A,2012,819,928,1026
1,A,2013,806,873,1038
2,A,2014,861,814,1009
3,A,2015,882,973,1036
4,A,2016,856,870,1042
5,B,2012,887,817,1008
6,B,2013,899,877,1041
7,B,2014,837,914,1009
8,B,2015,824,806,1002
9,B,2016,873,942,1013


Или так

In [72]:
sensors.set_index(['Group', 'Property']) \
       .stack() \
       .unstack('Property') \
       .rename_axis(['Group', 'Year'], axis='index') \
       .rename_axis(None, axis='columns') \
       .reset_index()

,Group,Year,Flow,Pressure,Temperature
0,A,2012,819,928,1026
1,A,2013,806,873,1038
2,A,2014,861,814,1009
3,A,2015,882,973,1036
4,A,2016,856,870,1042
5,B,2012,887,817,1008
6,B,2013,899,877,1041
7,B,2014,837,914,1009
8,B,2015,824,806,1002
9,B,2016,873,942,1013


# Подготовка данных, когда несколько наблюдений хранятся в одной таблице

In [73]:
movie = pd.read_csv('/content/drive/My Drive/6231-010402D/3сем/DV/movie_altered.csv')
movie.head()

,title,rating,year,duration,director_1,director_fb_likes_1,actor_1,actor_2,actor_3,actor_fb_likes_1,actor_fb_likes_2,actor_fb_likes_3
0,Avatar,PG-13,2009.0,178.0,James Cameron,0.0,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Pirates of the Caribbean: At World's End,PG-13,2007.0,169.0,Gore Verbinski,563.0,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Spectre,PG-13,2015.0,148.0,Sam Mendes,0.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,The Dark Knight Rises,PG-13,2012.0,164.0,Christopher Nolan,22000.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN,Doug Walker,131.0,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [74]:
movie.insert(0, 'id', np.arange(len(movie)))
movie.head()

,id,title,rating,year,duration,director_1,director_fb_likes_1,actor_1,actor_2,actor_3,actor_fb_likes_1,actor_fb_likes_2,actor_fb_likes_3
0,0,Avatar,PG-13,2009.0,178.0,James Cameron,0.0,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,1,Pirates of the Caribbean: At World's End,PG-13,2007.0,169.0,Gore Verbinski,563.0,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,2,Spectre,PG-13,2015.0,148.0,Sam Mendes,0.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,3,The Dark Knight Rises,PG-13,2012.0,164.0,Christopher Nolan,22000.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN,Doug Walker,131.0,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [75]:
stubnames = ['director', 'director_fb_likes', 'actor', 'actor_fb_likes']
movie_long = pd.wide_to_long(movie,
                                 stubnames=stubnames,
                                 i='id',
                                 j='num',
                                 sep='_').reset_index()
movie_long['num'] = movie_long['num'].astype(int)
movie_long.head(9)

,id,num,year,title,rating,duration,director,director_fb_likes,actor,actor_fb_likes
0,0,1,2009.0,Avatar,PG-13,178.0,James Cameron,0.0,CCH Pounder,1000.0
1,1,1,2007.0,Pirates of the Caribbean: At World's End,PG-13,169.0,Gore Verbinski,563.0,Johnny Depp,40000.0
2,2,1,2015.0,Spectre,PG-13,148.0,Sam Mendes,0.0,Christoph Waltz,11000.0
3,3,1,2012.0,The Dark Knight Rises,PG-13,164.0,Christopher Nolan,22000.0,Tom Hardy,27000.0
4,4,1,NaN,Star Wars: Episode VII - The Force Awakens,NaN,NaN,Doug Walker,131.0,Doug Walker,131.0
5,5,1,2012.0,John Carter,PG-13,132.0,Andrew Stanton,475.0,Daryl Sabara,640.0
6,6,1,2007.0,Spider-Man 3,PG-13,156.0,Sam Raimi,0.0,J.K. Simmons,24000.0
7,7,1,2010.0,Tangled,PG,100.0,Nathan Greno,15.0,Brad Garrett,799.0
8,8,1,2015.0,Avengers: Age of Ultron,PG-13,141.0,Joss Whedon,0.0,Chris Hemsworth,26000.0


In [76]:
movie_table = movie_long[['id','title', 'year', 'duration', 'rating']]
director_table = movie_long[['id', 'director', 'num', 'director_fb_likes']]
actor_table = movie_long[['id', 'actor', 'num', 'actor_fb_likes']]

In [77]:
movie_table.head(9)

,id,title,year,duration,rating
0,0,Avatar,2009.0,178.0,PG-13
1,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
2,2,Spectre,2015.0,148.0,PG-13
3,3,The Dark Knight Rises,2012.0,164.0,PG-13
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN
5,5,John Carter,2012.0,132.0,PG-13
6,6,Spider-Man 3,2007.0,156.0,PG-13
7,7,Tangled,2010.0,100.0,PG
8,8,Avengers: Age of Ultron,2015.0,141.0,PG-13


In [78]:
director_table.head(9)

,id,director,num,director_fb_likes
0,0,James Cameron,1,0.0
1,1,Gore Verbinski,1,563.0
2,2,Sam Mendes,1,0.0
3,3,Christopher Nolan,1,22000.0
4,4,Doug Walker,1,131.0
5,5,Andrew Stanton,1,475.0
6,6,Sam Raimi,1,0.0
7,7,Nathan Greno,1,15.0
8,8,Joss Whedon,1,0.0


In [79]:
actor_table.head(9)

,id,actor,num,actor_fb_likes
0,0,CCH Pounder,1,1000.0
1,1,Johnny Depp,1,40000.0
2,2,Christoph Waltz,1,11000.0
3,3,Tom Hardy,1,27000.0
4,4,Doug Walker,1,131.0
5,5,Daryl Sabara,1,640.0
6,6,J.K. Simmons,1,24000.0
7,7,Brad Garrett,1,799.0
8,8,Chris Hemsworth,1,26000.0


In [80]:
movie_table = movie_table.drop_duplicates().reset_index(drop=True)
director_table = director_table.dropna().reset_index(drop=True)
actor_table = actor_table.dropna().reset_index(drop=True)

In [81]:
movie_table.head()

,id,title,year,duration,rating
0,0,Avatar,2009.0,178.0,PG-13
1,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
2,2,Spectre,2015.0,148.0,PG-13
3,3,The Dark Knight Rises,2012.0,164.0,PG-13
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN


In [82]:
director_table.head()

,id,director,num,director_fb_likes
0,0,James Cameron,1,0.0
1,1,Gore Verbinski,1,563.0
2,2,Sam Mendes,1,0.0
3,3,Christopher Nolan,1,22000.0
4,4,Doug Walker,1,131.0


In [83]:
movie.memory_usage(deep=True).sum()

2300682

In [84]:
movie_table.memory_usage(deep=True).sum() + \
director_table.memory_usage(deep=True).sum() + \
actor_table.memory_usage(deep=True).sum()

2607442

In [85]:
director_cat = pd.Categorical(director_table['director'])
director_table.insert(1, 'director_id', director_cat.codes)

actor_cat = pd.Categorical(actor_table['actor'])
actor_table.insert(1, 'actor_id', actor_cat.codes)

director_table.head()

,id,director_id,director,num,director_fb_likes
0,0,922,James Cameron,1,0.0
1,1,794,Gore Verbinski,1,563.0
2,2,2020,Sam Mendes,1,0.0
3,3,373,Christopher Nolan,1,22000.0
4,4,600,Doug Walker,1,131.0


In [86]:
actor_table.head()

,id,actor_id,actor,num,actor_fb_likes
0,0,824,CCH Pounder,1,1000.0
1,1,2971,Johnny Depp,1,40000.0
2,2,1076,Christoph Waltz,1,11000.0
3,3,5859,Tom Hardy,1,27000.0
4,4,1615,Doug Walker,1,131.0


In [87]:
director_associative = director_table[['id', 'director_id', 'num']]
dcols = ['director_id', 'director', 'director_fb_likes']
director_unique = director_table[dcols].drop_duplicates().reset_index(drop=True)
director_associative.head()

,id,director_id,num
0,0,922,1
1,1,794,1
2,2,2020,1
3,3,373,1
4,4,600,1


In [88]:
director_unique.head()

,director_id,director,director_fb_likes
0,922,James Cameron,0.0
1,794,Gore Verbinski,563.0
2,2020,Sam Mendes,0.0
3,373,Christopher Nolan,22000.0
4,600,Doug Walker,131.0


In [89]:
actor_associative = actor_table[['id', 'actor_id', 'num']]
acols = ['actor_id', 'actor', 'actor_fb_likes']
actor_unique = actor_table[acols].drop_duplicates().reset_index(drop=True)
actor_associative.head()

,id,actor_id,num
0,0,824,1
1,1,2971,1
2,2,1076,1
3,3,5859,1
4,4,1615,1


In [90]:
actor_unique.head()

,actor_id,actor,actor_fb_likes
0,824,CCH Pounder,1000.0
1,2971,Johnny Depp,40000.0
2,1076,Christoph Waltz,11000.0
3,5859,Tom Hardy,27000.0
4,1615,Doug Walker,131.0


In [91]:
movie_table.memory_usage(deep=True).sum() + \
director_associative.memory_usage(deep=True).sum() + \
director_unique.memory_usage(deep=True).sum() + \
actor_associative.memory_usage(deep=True).sum() + \
actor_unique.memory_usage(deep=True).sum()

1816042

In [92]:
movie_table.head()

,id,title,year,duration,rating
0,0,Avatar,2009.0,178.0,PG-13
1,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13
2,2,Spectre,2015.0,148.0,PG-13
3,3,The Dark Knight Rises,2012.0,164.0,PG-13
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN


In [93]:
actors = actor_associative.merge(actor_unique, on='actor_id') \
                          .drop('actor_id', 1) \
                          .pivot_table(index='id', columns='num', aggfunc='first')

actors.columns = actors.columns.get_level_values(0) + '_' + \
                 actors.columns.get_level_values(1).astype(str)

directors = director_associative.merge(director_unique, on='director_id') \
                          .drop('director_id', 1) \
                          .pivot_table(index='id', columns='num', aggfunc='first')

directors.columns = directors.columns.get_level_values(0) + '_' + \
                    directors.columns.get_level_values(1).astype(str)

<ipython-input-93-b300c67178d7>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .drop('actor_id', 1) \
<ipython-input-93-b300c67178d7>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  .drop('director_id', 1) \


In [94]:
actors.head()

,actor_1,actor_2,actor_3,actor_fb_likes_1,actor_fb_likes_2,actor_fb_likes_3
id,,,,,,
0,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN


In [95]:
directors.head()

,director_1,director_fb_likes_1
id,,
0,James Cameron,0.0
1,Gore Verbinski,563.0
2,Sam Mendes,0.0
3,Christopher Nolan,22000.0
4,Doug Walker,131.0


In [96]:
movie2 = movie_table.merge(directors.reset_index(), on='id', how='left') \
                    .merge(actors.reset_index(), on='id', how='left')

In [97]:
movie2.head()

,id,title,year,duration,rating,director_1,director_fb_likes_1,actor_1,actor_2,actor_3,actor_fb_likes_1,actor_fb_likes_2,actor_fb_likes_3
0,0,Avatar,2009.0,178.0,PG-13,James Cameron,0.0,CCH Pounder,Joel David Moore,Wes Studi,1000.0,936.0,855.0
1,1,Pirates of the Caribbean: At World's End,2007.0,169.0,PG-13,Gore Verbinski,563.0,Johnny Depp,Orlando Bloom,Jack Davenport,40000.0,5000.0,1000.0
2,2,Spectre,2015.0,148.0,PG-13,Sam Mendes,0.0,Christoph Waltz,Rory Kinnear,Stephanie Sigman,11000.0,393.0,161.0
3,3,The Dark Knight Rises,2012.0,164.0,PG-13,Christopher Nolan,22000.0,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,27000.0,23000.0,23000.0
4,4,Star Wars: Episode VII - The Force Awakens,NaN,NaN,NaN,Doug Walker,131.0,Doug Walker,Rob Walker,NaN,131.0,12.0,NaN
